<a href="https://colab.research.google.com/github/deejay14/Jhakas-NewsWala/blob/main/Clickstream%20and%20Users_vs_artciles_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df=pd.read_csv('gdrive/My Drive/final_fixed_scrapped.csv')

In [21]:
def user_vs_article(df,users_no): ## df1 - dataframe of doc vs topic, no. of users
    #returns the clickstream data and user vs articles matrix
    import secrets
    from sklearn.mixture import GaussianMixture
    #df=pd.read_csv('gdrive/My Drive/final_fixed_scrapped.csv')
    df1=pd.read_csv('gdrive/My Drive/final scrapped/doc_vs_topic.csv')
    df2=pd.read_csv('gdrive/My Drive/final scrapped/topicwise_indices.csv')
    list=df['totalwords'].tolist()
    time=[]
    for i in range(7843):
        time.append(round((60*(list[i]/200)),2))##200 WPM
    df['average_reading_time(sec)']=time ##generating the avg. time column
    counts=df['average_reading_time(sec)'].value_counts()
    data =np.column_stack((counts.axes[0].tolist(),counts.tolist()))
 
# training gaussian mixture model 
    
    gmm = GaussianMixture(n_components=12)
    model=gmm.fit(data)
    #empty topic lists-1,6,8,9
 
    topic0=df2.loc[0].tolist()[1:121]    
    topic2=df2.loc[1].tolist()[1:2746]
    topic3=df2.loc[2].tolist()[1:1000]
    topic4=df2.loc[3].tolist()[1:1222]
    topic5=df2.loc[4].tolist()[1:34]
    topic7=df2.loc[5].tolist()[1:2463]
    topic10=df2.loc[6].tolist()[1:264]
    topic0=[int(x) for x in topic0]
    topic2=[int(x) for x in topic2]
    topic3=[int(x) for x in topic3]
    topic4=[int(x) for x in topic4]
    topic5=[int(x) for x in topic5]
    topic7=[int(x) for x in topic7]
    topic10=[int(x) for x in topic10]
     
 
 
    ## Clickstream generator lists
    lz=[]## reference list containing the no. of sessions of each user.
    for i in range(users_no):##user id column creation
        flag = True
        while flag:
              s = np.random.poisson(4, 1)
              if s[0]> 0:
                 lz.append(s[0])
                 flag=False
        
              else: flag=True
    user_id=[]## user_id creation
    #print(lz)
    k=1
    for i in range(len(lz)):
        for j in range(lz[i]*10):
            user_id.append(k)
        k=k+1
    rank=[] #rank column creation
    t=[1,2,3,4,5,6,7,8,9,10]
    for i in range(len(lz)):
        for j in range(lz[i]):
            rank=rank+t  
 
 
    q=[]## click list column creation
    for i in range(len(lz)):
        for j in range(lz[i]):
           r=np.random.poisson(5, 1)## poisson distribution determines the no. of articles read in each session.
           t=np.random.binomial(1,0.5,size=10)
           min=np.minimum(r,sum(t))
            #print(min)
           j=0
           a=0
           while j < min:
                if t[a]==1:
                   j=j+1
                   a=a+1
                else: a=a+1
 
           t_=t[0:a]
           t_=t_.tolist()
 
           for i in range(10-a):
               t_=t_ + [0]
           q=q+t_
    click=[]
    read_time=[]
    for i in range(len(q)):
       if q[i]==0:
          click.append('No')
       else: click.append('Yes')
    for i in range(len(q)):
        if q[i]==0:
           read_time.append('NaN')
        else: read_time.append(abs(model.sample(1)[0][0][0]))            
#session id
    sessionid=[]
 
    for j in lz:## session_id creation
        for k in range(j):
            t=k+1
            t_=[t for i in range(10)]
            sessionid=sessionid+t_  
    article_id=[]## article_id creation
    for i in range(len(lz)):
       for j in range(lz[i]):## distributing 10 articles from different topics.
          for p1 in range(2):
              article_id.append(secrets.choice(topic2))
          for p2 in range(2):
              article_id.append(secrets.choice(topic7))
          for p3 in range(2):
              article_id.append(secrets.choice(topic4))    
          article_id.append(secrets.choice(topic3))
          article_id.append(secrets.choice(topic10))
          article_id.append(secrets.choice(topic0))
          article_id.append(secrets.choice(topic5)) 
 
    t_spent=read_time
    article_id_list=article_id 
    #print(t_spent)
     
     
    click_stream = pd.DataFrame(    {'User_ID': user_id,
     'Session_ID':sessionid ,
     'Article_ID': article_id,
     'Article_rank': rank,
     'Click': click,
     'Time_spent': read_time
                          })     
    print(click_stream)
    ## User vs article matrix generator
 
    t_spent=read_time
    article_id_list=article_id
    user_vs_doc=np.zeros((1000,7843))
    u=0
    j0=0
#print(lz)
    for i in lz:  ##Implicit ratings generation
       for j in range(i*10):
           j0=sum(lz[0:u])*10+j
            #print(j0)
           index=article_id_list[j0]
           if type(t_spent[j0])==str:
               user_vs_doc[u][index]=0.0
          #print(index)  
           else: user_vs_doc[u][index] = t_spent[j0]/time[index]
        #print(user_vs_doc[u][index])
       u=u+1
    for i in range(users_no):
        for j in range(7843):
            if user_vs_doc[i][j]>1:
               user_vs_doc[i][j]=1.0
    #return(user_vs_doc)
 
    columns=[i for i in range(7843)]   
    index=[j for j in range(users_no)]
    df_user_vs_doc = pd.DataFrame(user_vs_doc, columns =columns, index = index) 
    print(df_user_vs_doc)

In [22]:
user_vs_article(df,1000)

       User_ID  Session_ID  Article_ID  Article_rank Click Time_spent
0            1           1        1097             1    No        NaN
1            1           1        2034             2   Yes    309.269
2            1           1        2205             3    No        NaN
3            1           1        2772             4    No        NaN
4            1           1        2302             5    No        NaN
...        ...         ...         ...           ...   ...        ...
40125     1000           5        3724             6   Yes    195.266
40126     1000           5        4702             7    No        NaN
40127     1000           5        6659             8    No        NaN
40128     1000           5        2074             9   Yes    123.554
40129     1000           5        7360            10    No        NaN

[40130 rows x 6 columns]
     0     1     2     3         4     5     ...  7837  7838  7839  7840  7841  7842
0     0.0   0.0   0.0   0.0  0.000000   0.0  ... 